## このノートブックの内容：
- エクセルにまとまっている質問それぞれに対して、指定したアシスタントの API を実行
- 結果をエクセルにまとめて出力する

> - API の作り方・叩き方の参考資料：https://www.notion.so/API-5da43a6fc4e7413bb7c1744ebfba1c3d

## 0. 事前設定

### 0.1. env の設定
- 上記資料などを参考にして、API Key と endpoint id を取得します。
- 取得したら、それらを .env ファイルに書き込みます

    ```py
    # .env にて、以下のように設定
    ASSISTANT_API_KEY="xxx"
    ASSISTANT_ENDPOINT_ID="xxx"
    ```

### 0.2. 質問票エクセルの用意
- 後述のコードは、`question_list.xlsx`のように、「質問」というカラムに質問がまとまっていることが前提になっています。
    - `question_list.xlsx`をよしなにいじって質問を設定すると簡単かなと思います
- 逆に、そこさえ問題なければ、質問がいくつあっても、他にカラムがあっても、処理は行われるはずです。

## 1. import & 環境変数用意

In [ ]:
import json
import os

from dotenv import load_dotenv
import pandas as pd
import requests

load_dotenv("../.env")

## 2. 関数定義

In [ ]:
def call_assistant(question: str) -> tuple[str, str]:
    """質問を入力として、アシスタントの API に回答させ、回答と参照元を返す

    Args:
        question (str): 質問

    Returns:
        tuple[str, str]: 回答, 参照元
    """
    print(f"Processing: {question}")
    url = f"https://chat.neoai.jp/api/endpoints/{os.environ.get('ASSISTANT_ENDPOINT_ID')}/chat/completions"
    headers = {"Authorization": f"Bearer {os.environ.get('ASSISTANT_API_KEY')}", "Content-Type": "application/json"}
    data = {"messages": [{"role": "user", "content": f"{question}"}], "stream": False}

    try:
        response = requests.post(url, headers=headers, data=json.dumps(data)).json()
    except Exception as e:
        print(e)
        return None, None

    # 辞書で帰ってくる参照元をテキストに変換
    citation = "\n\n".join(
        f"参照{i}: {d['chunk_name']}\n{d['content']}" for i, d in enumerate(response["data_points"], 1)
    )

    return response["content"], citation


def fill_out_question_sheet(file_path):
    """質問シートを読み込み、アシスタントに質問を投げて回答を埋める

    Args:
        file_path (_type_): ファイルパス

    Raises:
        ValueError: サポートされていないファイル形式です。.xlsx または .csv ファイルを使用してください。
    """
    _, ext = os.path.splitext(file_path)

    if ext not in [".xlsx", ".csv"]:
        raise ValueError("サポートされていないファイル形式です。.xlsx または .csv ファイルを使用してください。")

    read_func = pd.read_excel if ext == ".xlsx" else pd.read_csv
    sheet = read_func(file_path)

    sheet[["回答", "参照元"]] = sheet["質問"].apply(lambda x: pd.Series(call_assistant(x)))

    output_path = f"{os.path.splitext(file_path)[0]}_result{ext}"
    save_func = sheet.to_excel if ext == ".xlsx" else sheet.to_csv
    save_func(output_path, index=False)

## 3. エクセルファイルの処理（シートの質問を全て処理して、エクセルファイルにまとめて出力）

In [ ]:
# ここは好きに変えてください
PATH = "question_list.xlsx"

try:
    fill_out_question_sheet(PATH)
    print("処理が完了しました。")
except Exception as e:
    print(f"エラーが発生しました: {e}")